# Семинар 18

# Линейное программирование. Прямой метод внутренней точки

## На прошлом семинаре...

- Постановки задачи линейного программирования
- Примеры приложений
- Симплекс-метод и его недостатки

## Обзор существующих библиотек для решения задачи линейного программирования

- Решение задачи линейного прграммирования стало полноценной технологией, поэтому обычно нужно вызвать правильную функцию на правильном языке и получить ответ. 
- Обзор можно найти [тут](https://en.wikipedia.org/wiki/Linear_programming#Solvers_and_scripting_.28programming.29_languages)
- Сравнение можно найти [тут](http://prod.sandia.gov/techlib/access-control.cgi/2013/138847.pdf)

## Немного истории

- В 1979 г. Л. Хачиян [предложил](http://www.mathnet.ru/links/18409a6ec8cd239985969a9e98c2f6ae/zvmmf5239.pdf) метод эллипсоидов и показал, что он решает любую задачу линейного программирования за **полиномиальное время**. Сообщение об этом появилось на [первой полосе New York Times 7 ноября 1979](http://www.nytimes.com/1979/11/07/archives/a-soviet-discovery-rocks-world-of-mathematics-russians-surprise.html?_r=0)
- Однако сложность метода эллипсоидов $O(n^6 L^2)$, и он проигрывал симплекс-методу при решении реальных задач
- В 1984 г. Н. Кармаркар [предложил](http://retis.sssup.it/~bini/teaching/optim2010/karmarkar.pdf) другой полиномиальный метод решения задачи линейного программирования, который был значительно быстрее метода эллипсоидов, а именно $O(n^{3.5} L^2)$.  
- И метод эллипсоидов, и метод Кармаркара относятся к прямо-двойственным методам или методам внутренней точки, которые будут освещены далее...

## Альтернатива симплекс-методу

- Симплекс-метод основан на подходе поиска решения среди вершин
- Однако можно искать решение по-другому: двигаться по точкам внутри области к одной из вершин - решению задачи
- Поэтому такие методы называют *методами внутренней точки* 

## "Дёшево, но много" vs. "дорого, но мало"

- Один из основных водоразделов в методах оптимизации
- Линейное программирование решается за полиномиальное время, эксплуатируя стратегию "дорого, но мало"
- Далее будут показаны задачи и методы, которые работают по стратегии "дёшево, но много"

## Двойственность в задаче линейного программирования

**Теорема.**

- Если прямая (двойственная) задачи имеет конечное решение, то конечное решение имеет и двойственная (прямая).
- Если прямая (двойственная) задача неограничена, то допустимое множество двойственной (прямой) задачи пусто.

## Идея методов внутренней точки

Исходная задача
\begin{align*}
&\min_x c^{\top}x \\
\text{s.t. } & Ax = b\\
& x_i \geq 0, \; i = 1,\dots, n
\end{align*}

Аппроксимированная задача
\begin{align*}
&\min_x c^{\top}x {\color{red}{- \mu \sum\limits_{i=1}^n \ln x_i}} \\
\text{s.t. } & Ax = b\\
\end{align*}
для некоторого $\mu > 0$

### Барьерная функция

**Определение.** Функция $B(x, \mu) = -\mu\ln x$ называется *барьерной* для задачи с ограничением $x \geq 0$.

Более подробно о таких функциях будет рассказано в контексте нелинейной условной оптимизации...

### Что произошло?

- Сделали из линейной задачу нелинейную
- Перенесли ограничение типа неравенства в целевую функцию
- Ввели дополнительный параметр $\mu$

### Почему это хорошо?

Переход к задаче с ограничениями типа равенств $\to$ упрощение условий оптимальности, в частности

- Исключено требование дополняющей нежёсткости
- Исключено условие неотрицательности множителя Лагранжа для ограничения типа неравенства

### Условия оптимальности

- Лагранжиан: $L = c^{\top}x - \mu\sum\limits_{i=1}^n \ln x_i + \lambda^{\top}(Ax - b)$
- Стационарная точка $L$: 
$$
c - \mu X^{-1}e + A^{\top}\lambda = 0,
$$
где $X = \mathrm{diag}(x_1, \dots, x_n)$ и $e = [1, \dots, 1]$
- Ограничение типа равенства: $Ax = b$

Пусть $s = \mu X^{-1}e$, тогда условия оптимальности можно переписать так:
- $A^{\top}\lambda + c - s = 0 $
- $Xs = {\color{red}{\mu e}}$
- $Ax = b$

Также $x > 0 \Rightarrow s > 0$ 

## Сравнение с условиями оптимальности для исходной задачи

- Лагранжиан: $L = c^{\top}x + \lambda^{\top}(Ax - b) - s^{\top}x$
- Условие стационарности: $c + A^{\top}\lambda - s = 0$
- Допустимость прямой задачи: $Ax = b, \; x \geq 0$
- Допустимость двойственной: $s \geq 0$
- Условие дополняющей нежёсткости: $s_ix_i = 0$

### После упрощения

- $A^{\top}\lambda + c - s = 0$
- $Ax = b$
- $Xs = {\color{red}{0}}$
- $x \geq 0, \; s \geq 0$

## Вывод

- Введение барьерной функции c множителем $\mu$ эквивалентно релаксации условий дополняющей нежёсткости на параметр $\mu$
- При $\mu \to 0$ решения задач совпадают!
- Идея: итеративно решать задачи с барьерной функцией, уменьшая $\mu$. Последовательность решений сойдётся к вершине симплекса по траектории из точек, лежащих внутри симплекса.

### Общая схема
```python
def GeneralInteriorPointLP(c, A, b, x0, mu0, rho, tol):
    x = x0
    mu = mu0
    e = np.ones(c.shape[0])
    while True:
        primal_var, dual_var = StepInsideFeasibleSet(c, A, b, x, mu)
        mu *= rho
        if converge(primal_var, dual_var, c, A, b, tol) and mu < tol:
            break
    return x
```

## Как решать задачу с барьерной функцией?

- Прямой метод - следующий слайд
- Прямо-двойственный метод - через несколько недель

## Прямой метод

Вспомним исходную задачу:
\begin{align*}
&\min_x c^{\top}x - \mu \sum\limits_{i=1}^n \ln x_i \\
\text{s.t. } & Ax = b\\
\end{align*}

Идея: приблизим целевую функцию до второго порядка, как в методе Ньютона.

### Реализация
На $(k+1)$-ой итерации необходимо решить следующую задачу: 
\begin{align*}
&\min_p \frac{1}{2}p^{\top}Hp + g^{\top}p\\
\text{s.t. } & A(x_k + p) = b,\\
\end{align*}
где $H = \mu X^{-2}$ - гессиан, и $g = c - \mu X^{-1}e$ - градиент.

### Снова KKT

Выпишем условия ККТ для этой задачи
- $Hp + g + A^{\top}\lambda = 0$
- $Ap = 0$

или
$$\begin{bmatrix}
H & A^{\top}\\
A & 0
\end{bmatrix}
\begin{bmatrix}
p\\
\lambda
\end{bmatrix}
= 
\begin{bmatrix}
-g\\
0
\end{bmatrix}$$

Из первой строки:
$$
-\mu X^{-2}p + A^{\top}\lambda = c - \mu X^{-1}e
$$
$$
-\mu Ap + AX^{2}A^{\top}\lambda = AX^2c - \mu AXe
$$
$$
AX^{2}A^{\top}\lambda = AX^2c - \mu AXe
$$
Так как $X \in \mathbb{S}^n_{++}$ и $A$ полного ранга, то уравнение имеет единственное решение $\lambda^*$.

### Найдём направление $p$

$$
-\mu p + X^2A^{\top}\lambda^* = X^2c - \mu Xe = X^2c - \mu x
$$
$$
p = x + \frac{1}{\mu}X^2(A^{\top}\lambda^* - c)
$$

### Поиск шага $\alpha$

- Обычный линейный поиск с условиями достаточного убывания
- Условие $A(x_k + \alpha p) = b$ выполняется автоматически

### Псевдокод прямого барьерного метода

```python
def PrimalBarrierLP(c, A, b, x0, mu0, rho, tol):
    x = x0
    mu = mu0
    e = np.ones(x.shape[0])
    while True:
        p, lam = ComputeNewtonDirection(c, x, A, mu)
        alpha = line_search(p, mu, c, x)
        x = x + alpha * p
        mu = rho * mu
        if mu < tol and np.linalg.norm(x.dot(c - A.T.dot(lam)) - mu * e) < tol:
            break
    return x
```

### Комментарии

- Было показано, что прямой метод эквивалентен методу Кармаркара
- Использует информацию только о прямой задаче
- Начальное приближение должно лежать в допустимом множестве - отдельная задача

## Резюме 

1. История развития теории линейного программирования и текущее состояние
2. Концепция методов внутренней точки
3. Прямой метод решения задачи линейного программирования